### In this Notebook, we will consider three learning models to predict Sales for each Item in each Outlet

### Namely, the three models are:
1. Linear Regression model.
2. KNN regression model.
3. Decision tree regression model.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split, validation_curve, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from statsmodels.regression.linear_model import OLS, OLSResults, add_constant
from sklearn.externals import joblib

%matplotlib inline

In [ ]:
train = pd.read_csv('train_engineered.csv')
test = pd.read_csv('test_engineered.csv')

In [ ]:
features = train.drop(columns=['Item_Identifier', 'Outlet_Identifier','Item_Outlet_Sales'])
response = train[['Item_Outlet_Sales']]

## Baseline Model - Model 0

### In this model, the predicted sales for each item is the mean sales for this item across all Outlets.

In [ ]:
mean_sales_type = train.pivot_table(values= 'Item_Outlet_Sales', index= 'Item_Identifier', aggfunc='mean')

In [ ]:
pred0 = test.loc[:,'Item_Identifier'].apply(lambda x: mean_sales_type.loc[x])

In [ ]:
model0 = pd.concat([test.Item_Identifier, test.Outlet_Identifier, pred0],axis=1)

In [ ]:
model0.to_csv('model0.csv', index=False)

## K-Fold Cross Validation

In [ ]:
kf = KFold(n_splits = 10)

## Linear Regression - Model 1

### Including all features in training.

In [ ]:
lr = LinearRegression()

In [ ]:
RMSE_lr = np.round(np.sqrt(np.mean(np.abs(cross_val_score(lr,features,y= response,scoring= 'neg_mean_squared_error', cv=kf)))),3)

In [ ]:
R_Sq_lr = np.round(np.mean(cross_val_score(lr,features,y= response,scoring= 'r2', cv=kf)),3)

In [ ]:
print('RMSE: {}, R-Sq: {}'.format(RMSE_lr, R_Sq_lr))

Exporting submission file.

In [ ]:
y_pred_sub = lr.predict(test.drop(columns=['Item_Identifier','Outlet_Identifier']))

In [ ]:
y_pred_df = pd.DataFrame(y_pred_sub,columns=['Item_Outlet_Sales'])

In [ ]:
LR_model = pd.concat([test.Item_Identifier, test.Outlet_Identifier, y_pred_df],axis=1)

In [ ]:
LR_model.to_csv('Linear_Regression_Model.csv', index=False)

## Linear Regression - Selected features - Model 2

### We will execlude features that are highly correlated and check if RMSE and R-Sq are improved.

In [ ]:
features[['Item_Visibility','Item_MRP','visibility_ratio','Item_MeanPrice_Ratio', 'specific_price']].corr()

In [ ]:
selected_features = features.drop(columns= ['Item_Weight','Item_Visibility','specific_price','visibility_ratio'])

In [ ]:
lr_sf = LinearRegression()

In [ ]:
RMSE_lr_sf = np.round(np.sqrt(np.mean(np.abs(cross_val_score(lr_sf,selected_features,y= response,scoring= 'neg_mean_squared_error', cv=kf)))),3)

In [ ]:
R_Sq_lr_sf = np.round(np.mean(cross_val_score(lr_sf,selected_features,y= response,scoring= 'r2', cv=kf)),3)

In [ ]:
print('RMSE: {}, R-Sq: {}'.format(RMSE_lr_sf, R_Sq_lr_sf))

## KNN - Model 3

In [ ]:
knn = KNeighborsRegressor()

### Grid Search Cross Validation is used to optimize the paramters of KNN. 

In [ ]:
param_grid = dict({'n_neighbors':list(range(3,16)),'weights':['uniform','distance'],'algorithm':['ball_tree','kd_tree', 'brute'],'p':[1,2]})

In [ ]:
GSCV = GridSearchCV(knn, param_grid, scoring='neg_mean_squared_error', cv= kf)

In [ ]:
GSCV.fit(features,y = response.values.reshape(-1))

In [ ]:
joblib.dump(GSCV,'knn_fitted.joblib')

In [ ]:
knn_fitted = joblib.load('knn_fitted.joblib')

In [ ]:
knn_fitted.best_params_

In [ ]:
RMSE_knn = np.round(np.sqrt(np.mean(np.abs(cross_val_score(knn_fitted,features,y=response, scoring='neg_mean_squared_error',cv=kf)),3)

In [ ]:
R_Sq_knn = np.round(np.mean(cross_val_score(knn_fitted,selected_features,y= response,scoring= 'r2', cv=kf)),3)

In [ ]:
print('RMSE: {}, Adj R-Sq: {}'.format(RMSE_knn, R_Sq_knn))

Exporting submission file.

In [ ]:
y_pred_sub = GSCV.predict(test.drop(columns=['Item_Identifier','Outlet_Identifier']))

In [ ]:
y_pred_df = pd.DataFrame(y_pred_sub,columns=['Item_Outlet_Sales'])

In [ ]:
KNN_model = pd.concat([test.Item_Identifier, test.Outlet_Identifier, y_pred_df],axis=1)

In [ ]:
KNN_model.to_csv('KNN_Regression_Model.csv', index=False)

## Decision Tree - Model3

In [ ]:
dt = DecisionTreeRegressor(max_depth= 29, max_features = 31, min_samples_split = 412)

### Grid Search Cross Validation is used to optimize the paramters of Decision Tree. 

In [ ]:
param_grid = dict({'min_samples_split': np.arange(2,510,10).tolist(), 'max_features':list(range(1,features.shape[1])), 'max_depth': list(range(10,31))})

In [ ]:
GSCV_dt = GridSearchCV(dt, param_grid, scoring='neg_mean_squared_error',cv= kf)

In [ ]:
GSCV_dt.fit(features,y = response.values.reshape(-1))

In [ ]:
joblib.dump(GSCV_dt,'dt_fitted.joblib')

In [ ]:
dt_fitted = joblib.load('dt_fitted.joblib')

In [ ]:
dt_fitted.best_params_

In [ ]:
RMSE_dt = np.round(np.sqrt(np.abs(dt_fitted.best_score_)),3)

In [ ]:
y_pred = dt_fitted.predict(x_test)

In [ ]:
R_Sq_Adj_dt = np.round(np.sqrt(r_sq_adj(y_test, y_pred, features.shape[1])),3)

In [ ]:
print('RMSE: {}, Adj R-Sq: {}'.format(RMSE_dt, R_Sq_Adj_dt))

Exporting submission file.

In [ ]:
y_pred_sub = dt.predict(test.drop(columns=['Item_Identifier','Outlet_Identifier']))

In [ ]:
y_pred_df = pd.DataFrame(y_pred_sub,columns=['Item_Outlet_Sales'])

In [ ]:
DT_model = pd.concat([test.Item_Identifier, test.Outlet_Identifier, y_pred_df],axis=1)

In [ ]:
DT_model.to_csv('DecisionTree_Regression_Model.csv', index=False)

### Visualizing Decision Tree

In [ ]:
from graphviz import Source
from IPython.display import display
from IPython.display import SVG

In [ ]:
dt.fit(x_train,y_train)

In [ ]:
graph2 = Source(export_graphviz(dt, out_file= None, feature_names= features.columns.tolist(),filled = True, rounded= True))

In [ ]:
display(SVG(graph2.pipe(format='svg')))

# Model Comparison

In [ ]:
pd.DataFrame([[RMSE_lr, R_Sq_lr], [RMSE_knn, R_Sq_knn], [RMSE_dt, R_Sq_Adj_dt]],
             columns=['RMSE','Adjusted R-Sq'], 
             index=['Linear Regression','KNN','Decision Tree'])